In [ ]:
"""
Script responsável por realizar limpeza nos diff_hunks, deixando apenas o trecho modificado.
"""

ERROR: Could not find a version that satisfies the requirement tensorflow==2.5.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==2.5.0
  Using cached guesslang-2.2.1-py3-none-any.whl (2.5 MB)
INFO: pip is looking at multiple versions of guesslang to determine which version is compatible with other requirements. This could take a while.
  Using cached guesslang-2.2.0-py3-none-any.whl (2.5 MB)
  Using cached guesslang-2.0.3-py3-none-any.whl (2.1 MB)
  Using cached guesslang-2.0.1-py3-none-any.whl (2.1 MB)
  Using cached guesslang-2.0.0-py3-none-any.whl (13

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataset = pd.read_csv('<PATH_DATASET>')
dataset.head()

In [ ]:
import requests
import pandas as pd
import re
from time import sleep

In [ ]:
df = pd.read_csv('<PATH_DATASET>')

In [ ]:
def build_comment_url(func):
  def wrapper(row):
    pr_url = row['pull_request_url']
    comment_id = row['comment_id']
    pattern = r'[\w+:./-]*\/'

    base_url = re.findall(pattern, pr_url)[0]

    return func(f'{base_url}comments/{comment_id}')

  return wrapper

In [ ]:
def get_diff_hunk(func):
  def wrapper(comment_url):
    token = '<TOKEN>'
    headers = {
      'Authorization': 'token ' + token,
    }

    r = requests.get(comment_url, headers=headers)
    if r.status_code == 200:
      return func(r.json().get('diff_hunk'))
    else:
      print(f'{comment_url} - {r.status_code}')
      return func({'status_code': r.status_code, 'url': comment_url})

  return wrapper

In [ ]:
def remove_line_range(func):
  def wrapper(diff_hunk):
    pattern = r'@@.*?@@'
    return func(re.sub(pattern, '', diff_hunk) + '\n')

  return wrapper

In [ ]:
def filter_only_added_lines(func):
  def wrapper(diff_hunk):
    pattern = r'\+(.*?)\n'
    lines = re.findall(pattern, diff_hunk, re.MULTILINE)

    return func('\n'.join(lines))

  return wrapper

In [ ]:
@build_comment_url
@get_diff_hunk
@remove_line_range
@filter_only_added_lines
def clean_comments(diff_hunk):
  if isinstance(diff_hunk, str):
    pattern = r'(\""".*?\"""|\'\'\'.*?\'\'\'|"""[\s\S]*?"""|\'\'\'[\s\S]*?\'\'\'|/\*[\s\S]*?\*/|//.*?$)'
    return re.sub(pattern, '', diff_hunk)
  else:
    return diff_hunk

In [ ]:
from guesslang import Guess


guess = Guess()

name = guess.language_name("""
    % Quick sort

    -module (recursion).
    -export ([qsort/1]).

    qsort([]) -> [];
    qsort([Pivot|T]) ->
          qsort([X || X <- T, X < Pivot])
          ++ [Pivot] ++
          qsort([X || X <- T, X >= Pivot]).
""")

print(name)  # ⟶ Erlang

In [ ]:
df['cleaned_diff_hunk'] = df.apply(clean_comments, axis=1)
df.to_csv('<PATH_DATASET_CLEANED>', index=False)

In [ ]:
@build_comment_url
def get_diff_hunk(func):
  token = '<TOKEN>'
  headers = {
    'Authorization': 'token ' + token,
  }

  r = requests.get(comment_url, headers=headers)
  if r.status_code == 200:
    return func(r.json().get('diff_hunk'))
  else:
    print(f'{comment_url} - {r.status_code}')
    return func({'status_code': r.status_code, 'url': comment_url})

df['formatted_diff_hunk'] = df.apply(get_diff_hunk, axis=1)